In [2]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter

In [90]:
def build_qft_circuit(n_qubits: int) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    for k in range(0, n_qubits // 2):
        circ.SWAP(k, n_qubits - k - 1)
    return circ

In [3]:
from pytket.circuit import CircBox
from pytket.circuit import QControlBox

In [79]:
def build_phase_estimation_circuit(
    n_measurement_qubits: int, state_prep_circuit: Circuit, unitary_circuit: Circuit
) -> Circuit:
    # Define a Circuit with a measurement and prep register
    qpe_circ: Circuit = Circuit()
    n_state_prep_qubits = state_prep_circuit.n_qubits
    measurement_register = qpe_circ.add_q_register("m", n_measurement_qubits)
    state_prep_register = qpe_circ.add_q_register("p", n_state_prep_qubits)
    qpe_circ.add_circuit(state_prep_circuit, list(state_prep_register))

    # Create a controlled unitary with a single control qubit
    unitary_circuit.name = "U"
    controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)

    # Add Hadamard gates to every qubit in the measurement register
    for m_qubit in measurement_register:
        qpe_circ.H(m_qubit)

    # Add all (2**n_measurement_qubits - 1) of the controlled unitaries sequentially
    for m_qubit in range(n_measurement_qubits):
        control_index = n_measurement_qubits - m_qubit - 1
        control_qubit = [measurement_register[control_index]]
        for _ in range(2**m_qubit):
            qpe_circ.add_qcontrolbox(
                controlled_u_gate, control_qubit + list(state_prep_register)
            )

    # Finally, append the inverse qft and measure the qubits
    qft_box = CircBox(build_qft_circuit(n_measurement_qubits))
    inverse_qft_box = qft_box.dagger
    qpe_circ.add_circbox(inverse_qft_box, list(measurement_register))
    qpe_circ.measure_register(measurement_register, "c")
    return qpe_circ

In [5]:
prep_circuit = Circuit(1).X(0)  # prepare the |1> eigenstate of U1
input_angle = 0.73  # angle as number of half turns
unitary_circuit = Circuit(1).U1(input_angle, 0)  # Base unitary for controlled U ops
qpe_circ_trivial = build_phase_estimation_circuit(
    4, state_prep_circuit=prep_circuit, unitary_circuit=unitary_circuit
)
render_circuit_jupyter(qpe_circ_trivial)

In [83]:
prep_circ = Circuit(4)
render_circuit_jupyter(prep_circ)

In [84]:
from pytket.circuit import Op
from pytket.circuit import OpType

In [85]:
value=0.75
ControlZ=Op.create(OpType.Rz,value)

In [86]:
cx=QControlBox(ControlZ,n_controls=3,control_state=[0,1,2])

In [87]:
prep_circ.CX(3,2)
prep_circ.CX(3,1)
prep_circ.CX(3,0)
prep_circ.H(3)
prep_circ.add_gate(cx,[2,1,0,3])
prep_circ.H(3)
prep_circ.CX(3,0)
prep_circ.CX(3,1)
prep_circ.CX(3,2)

[CX q[3], q[2]; CX q[3], q[1]; CX q[3], q[0]; H q[3]; qif (q[2] = 0, q[1] = 1, q[0] = 1) Rz(0.75) q[3]; H q[3]; CX q[3], q[0]; CX q[3], q[1]; CX q[3], q[2]; ]

In [88]:
render_circuit_jupyter(prep_circ)

In [91]:
preparation_circuit = Circuit(4)# prepare the |1> eigenstate of U1
preparation_circuit.H(0)
preparation_circuit.H(1)
preparation_circuit.H(2)
preparation_circuit.H(3)

unitary_circuit = prep_circ  # Base unitary for controlled U ops
qpe_circ_trivial = build_phase_estimation_circuit(
    4, state_prep_circuit=prep_circ, unitary_circuit=unitary_circuit
)
render_circuit_jupyter(qpe_circ_trivial)

In [69]:
left_circ = Circuit(4)
left_circ.CX(3,2)
left_circ.CX(3,1)
left_circ.CX(3,0)
render_circuit_jupyter(left_circ)

In [70]:
right_circ = Circuit(4)
right_circ.CX(3,0)
right_circ.CX(3,1)
right_circ.CX(3,2)
render_circuit_jupyter(right_circ)

In [82]:
new_box = CircBox(left_circ)
prep_circ.add_circbox(new_box,[0, 1, 2, 3])
render_circuit_jupyter(prep_circ)

In [ ]:
control_qubit=1
unitary_circuit.name = "U"
controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)
prep_circ.add_qcontrolbox(
                controlled_u_gate, [measurement_register] + list(state_prep_register)
            )

In [ ]:
control_qubit=1
unitary_circuit.name = "U"
controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)
prep_circ.add_qcontrolbox(
                controlled_u_gate, [measurement_register] + list(state_prep_register)
            )

In [25]:
from pytket import Circuit

# Create a quantum circuit with 3 qubits
circuit = Circuit(3)

# Apply a Hadamard gate to qubit 0 to create a superposition
circuit.H(0)

# Add a controlled-X (CNOT) gate with qubit 0 as control and qubit 1 as target
circuit.CX(0, 1)

# For a more complex control structure, you can use a CCX (Toffoli) gate
# Here, qubits 0 and 1 are controls, and qubit 2 is the target
circuit.CCX(0, 1, 2)

# Visualize the circuit (this requires installing optional visualization packages)
render_circuit_jupyter(circuit)

In [31]:
from pytket import Circuit

# Create a quantum circuit with 3 qubits
circuit = Circuit(2,1)

# Apply a Hadamard gate to qubit 0 to create a superposition
circuit.H(0)

# Add a controlled-X (CNOT) gate with qubit 0 as control and qubit 1 as target
circuit.X(1, condition_bits=[0],condition_value=0)


# Visualize the circuit (this requires installing optional visualization packages)
render_circuit_jupyter(circuit)